In [1]:
import collections
import heapq
import itertools
import os
import shutil
import string

import numpy as np
import pandas as pd

In [2]:
class Problem(object):
    
    def __init__(self, problem_name, has_subtests=False):
        self.problem_name = problem_name
        self.test_id = 0
        self.has_subtests = has_subtests
        if os.path.exists(self.problem_name):
            shutil.rmtree(self.problem_name)
        os.makedirs(f'{self.problem_name}/input')
        os.makedirs(f'{self.problem_name}/output')
        
    def create_inputs(self, **kwargs):
        """Should return a list. Each element is a line in the input."""
        raise NotImplementedError
    
    def solve(self, inputs, **kwargs):
        """Should return a list. Each element is a line in the output."""
        raise NotImplementedError
    
    @staticmethod
    def save_to_file(data, file_path=None, **kwargs):
        if isinstance(data, list):
            data = ' '.join(str(x) for x in data)
        print(data, file=file_path, **kwargs)
    
    def create_single_test(self, inputs=None, **kwargs):
        n = 1
        if inputs is None:
            if self.has_subtests:
                if 'N' in kwargs:
                    n = kwargs['N']
                else:
                    n = np.random.randint(kwargs['N_MIN'], kwargs['N_MAX']+1)
                inputs = [n]
                outputs = []
                for _ in range(n):
                    inp = self.create_inputs(**kwargs)
                    outp = self.solve(inp)
                    inputs += inp
                    outputs += outp
            else:
                inputs = self.create_inputs(**kwargs)
                outputs = self.solve(inputs)
        else:
            outputs = self.solve(inputs, **kwargs)
            if self.has_subtests:
                inputs = [1] + inputs

        self.write_test(self.test_id, inputs, outputs)
        self.test_id += 1
        return (self.test_id-1, inputs, outputs)
    
    def create_multiple_tests(self, n, **kwargs):
        for i in range(n):
            yield self.create_single_test(**kwargs)
            
    def write_test(self, test_id, inputs, outputs):
        input_file = f'{self.problem_name}/input/input_{test_id:02d}.txt'
        output_file = f'{self.problem_name}/output/output_{test_id:02d}.txt'

        with open(input_file, 'w') as fp:
            for line in inputs:
                Problem.save_to_file(line, fp)

        with open(output_file, 'w') as fp:
            for line in outputs:
                Problem.save_to_file(line, fp)

In [3]:
class TopCoderProblem(Problem):
    
    def parse_inputs(self, inputs):
        return inputs
    
    def parse_outputs(self, outputs):
        return outputs
    
    def create_from_string(self, str_data):
        data = str_data.split('\n')
        for line in data:
            line = line.strip()
            if not line:
                continue
            line = line.replace('{', '[').replace('}', ']')
            parts = [eval(part) for part in line.split('\t\t')[:-1]]
            assert len(parts) == 2
            
            inputs = self.parse_inputs(parts[0])
            outputs = self.parse_outputs(parts[1])
            
            self.write_test(self.test_id, inputs, outputs)
            self.test_id += 1
            yield (self.test_id - 1, inputs, outputs)

In [4]:
!ls

algo-note.ipynb  codesignal	       interviewbit  README.md
codelearn	 create_problem.ipynb  leetcode      topcoder
